# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
#csv_file = os.path.join('..', 'WeatherPy', 'Output', 'ity.csv')
weather_data = "../WeatherPy/output_data/Cities.csv"

# Read the weather data and the study results
read_weather = pd.read_csv(weather_data)

# Combine the data into a single dataset
city_df = pd.DataFrame(read_weather)
city_df.head()

,City ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,Jamestown,42.0970,-79.2353,64.71,90,90,3.00,1627811429,US
1,1,Kapit,2.0167,112.9333,78.58,90,100,4.92,1627811329,MY
2,2,Atuona,-9.8000,-139.0333,77.23,64,85,13.82,1627811429,PF
3,3,Hobart,-42.8794,147.3294,44.06,74,20,4.00,1627811430,AU
4,4,Saskylakh,71.9167,114.0833,83.30,27,66,8.55,1627811391,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [9]:
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_df = city_df[(city_df['Max Temperature'] > 70) & (city_df['Max Temperature'] < 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] < 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]
ideal_df

,City ID,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Country
150,150,Luderitz,-26.6481,15.1594,78.87,23,0,6.38,1627811447,NaN
342,342,Cayenne,4.9333,-52.3333,76.08,91,0,3.44,1627811416,GF
362,362,Kalabo,-14.9700,22.6814,77.99,25,0,6.98,1627811472,ZM
371,371,Urumqi,43.8010,87.6005,79.75,23,0,6.71,1627811178,CN
378,378,Caucaia,-3.7361,-38.6531,76.03,74,0,9.22,1627811474,BR
397,397,Bowen,-20.0167,148.2333,70.02,76,0,8.01,1627811476,AU
515,515,Aykhal,66.0000,111.5000,78.80,27,0,5.23,1627811490,RU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_df

In [12]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
     # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 150: Luderitz.
Closest hotel is LÜDERITZ NEST HOTEL.
------------
Retrieving Results for Index 342: Cayenne.
Closest hotel is Hôtel Le Dronmi.
------------
Retrieving Results for Index 362: Kalabo.
Closest hotel is Sikakubete Guest House Kalabo Town Council.
------------
Retrieving Results for Index 371: Urumqi.
Closest hotel is Hua Ling Grand Hotel Urumqi.
------------
Retrieving Results for Index 378: Caucaia.
Closest hotel is M.O.S. Logística - Filial Fortaleza.
------------
Retrieving Results for Index 397: Bowen.
Closest hotel is Castle Motor Lodge Motel.
------------
Retrieving Results for Index 515: Aykhal.
Missing field/result... skipping.
------------


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))